In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt

In [2]:
data_generation_process = "SPO_Data_Generation"
data_generation_process = "DDR_Data_Generation"

# Parameters

In [3]:
# import pyepo
# generate data
grid = (3,3) # grid size
num_train = 100 # number of training data
num_feat = 5 # size of feature
num_test = 10000
deg = 1.0 # polynomial degree
e = 1.0 # scale of normal std or the range of uniform. For the error term

lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
num_nodes = grid[0]*grid[0]
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 7

In [4]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data/Test0401_' + data_generation_process + "/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("grandparent_directory:", grandparent_directory)
print("DataPath:", DataPath)
DataPath = DataPath + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_coef_seed="+str(coef_seed)+"_same_W/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)

grandparent_directory: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Test0401_DDR_Data_Generation/


# Generate Data

In [5]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,data_generation_process) 
    # print("W_star = ",W_star[0,:])
    np.random.seed(coef_seed)
    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}
    for iter in iteration_all:
        DataPath_iter = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_iter).mkdir(parents=True, exist_ok=True)
        # W_star = data_gen.generate_truth(DataPath_iter,lower, upper, p, d, iter,data_generation_process) 
        # #  ****** Data generation *********
        x_test_all[iter], c_test_all[iter], x_train_all[iter], c_train_all[iter], W_star_all[iter] = data_gen.generate_samples(iter,DataPath_iter,p, d, num_test, num_train, alpha, W_star, mis, num_test, 
                                data_generation_process, x_dist, e_dist, x_low, x_up, x_mean, x_var, bump) 
        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, W_star_all

# EPO(SPO+,...)

In [6]:
# from PYEPO import PyEPO_Method
# epo_runner = PyEPO_Method()
def Implement_EPO(DataPath,seed_all,batch_size,num_epochs,method_names,x_train_all,c_train_all,x_test_all,c_test_all,arcs,epo_runner):
    cost_EPO = {}
    for seed in seed_all:
        DataPath_seed = DataPath +"Seed="+str(seed)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        # print("*** seed = ",seed,": Run EPO ******")
        cost_EPO[seed] = epo_runner.run(method_names,DataPath_seed,batch_size,num_feat,grid,num_epochs,\
                                        x_train_all[seed],c_train_all[seed],x_test_all[seed],c_test_all[seed],arcs)
    return cost_EPO

# Obtain DDR estimation

### define network

In [7]:
def _getArcs(grid):
    arcs = []
    for i in range(grid[0]):
        # edges on rows
        for j in range(grid[1] - 1):
            v = i * grid[1] + j
            arcs.append((v, v + 1))
        # edges in columns
        if i == grid[0] - 1:
            continue
        for j in range(grid[1]):
            v = i * grid[1] + j
            arcs.append((v, v + grid[1]))

    arc_index_mapping = {}
    for i in range(len(arcs)):
        arc = arcs[i]
        arc_index_mapping[arc] = i

    return arcs,arc_index_mapping

In [8]:
def solve_DDR(arcs,lamb,mu_fixed,num_nodes,x_train,c_train):
    
    N,p = x_train.shape
    N,d = c_train.shape

    # DDR
    m = Model("ddr")
    #m.setParam("DualReductions",0)
    m.setParam('OutputFlag', 0)

    W_ind = tuplelist( [(i,j) for i in range(d) for j in range(p)] )
    w0_ind = tuplelist( [i for i in range(d)])

    W_ddr = m.addVars(W_ind, lb=-GRB.INFINITY,name = "W" )
    w0_ddr = m.addVars(w0_ind, lb=-GRB.INFINITY,name = "W0" )
    alpha = m.addVars(N,num_nodes,lb=-GRB.INFINITY,name="alpha")
    expr_obj = 0
    err = []
    for n in range(N):
        cost_true_tem = c_train[n]
        expr_obj = expr_obj + alpha[n,num_nodes-1] - alpha[n,0]
        for ind in range(len(arcs)):
            cost_pred_tem = quicksum([W_ddr[ind,j] * x_train[n,j] for j in range(p)]) + w0_ddr[ind]
            err.append(cost_true_tem[ind] - cost_pred_tem)
            e = arcs[ind]
            j = e[1]
            i = e[0]
            # print("j = ",j,", i = ",i, ", e = ",e)
            m.addConstr(alpha[n,j] - alpha[n,i] >= -mu_fixed*cost_true_tem[ind] - (1-mu_fixed)*cost_pred_tem)

    m.setObjective(quicksum([err[k] * err[k] for k in range(len(err))])/N + lamb*(expr_obj)/N, GRB.MINIMIZE)
    m.optimize()
    
    W_DDR_rst = m.getAttr('x', W_ddr)
    w0_DDR_rst = m.getAttr('x', w0_ddr)
    W_ddr_val = []
    for i in range(d):
        W_ddr_val.append([W_DDR_rst[(i,j)] for j in range(p)])
    w0_ddr_val = [w0_DDR_rst[i] for i in range(d)]

    alpha_rst = m.getAttr('x', alpha)
    return w0_ddr_val,W_ddr_val,alpha_rst,m.ObjVal

# Out-of-sample performance

### Generate data

In [9]:
arcs,arc_index_mapping = _getArcs(grid)
num_arcs = len(arcs)
iteration_all = np.arange(0,10)
# obtain data
x_test_all, c_test_all, x_train_all, c_train_all, W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

### EPO performance

In [10]:
# # EPO, including SPO, PG, LTR
# batch_size = 20
# num_epochs = 100
# method_names = ["spo+","pg","ltr"]
# method_names = ["spo+"]
# from PYEPO import PyEPO_Method
# epo_runner = PyEPO_Method()

# cost_EPO_all = Implement_EPO(DataPath,[1],batch_size,num_epochs,method_names,x_train_all,c_train_all,x_test_all,c_test_all,arcs,epo_runner)
# print("SPO+Cost = ",np.nanmean(cost_EPO_all[1]["SPO"]))

## Oracle performance

In [11]:
from Peformance import performance_evaluation
perfs = performance_evaluation()
cost_Oracle_all = {}
for iter in iteration_all:
    if data_generation_process == "SPO_Data_Generation":
        cost_dem = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
        oracle_cost_pred = (cost_dem ** mis + 1).T
    if data_generation_process == "DDR_Data_Generation":
        cost_dem = (W_star_all[iter] @ x_test_all[iter].T) + bump
        oracle_cost_pred = (cost_dem ** mis).T
    cost_Oracle_all[iter] = perfs.compute_Oracel_Cost(arcs, grid,oracle_cost_pred,c_test_all[iter])
    # cost_Oracle_all[iter] = perfs.compute_Cost_with_Prediction(arcs,np.ones(num_arcs)*bump, W_star_all[iter],grid,c_test_all[iter],x_test_all[iter])

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13


### OLS performance

In [12]:
from OLS import ols_method
ols_method_obj = ols_method()
W_ols_all = {}; w0_ols_all = {}; t_ols_all = {}; obj_ols_all = {}
cost_OLS_all = {}
for iter in iteration_all:
    # compute OLS performance
    W_ols_all[iter], w0_ols_all[iter], t_ols_all[iter], obj_ols_all[iter] = ols_method_obj.ols_solver("",x_train_all[iter], c_train_all[iter])
    cost_dem = (W_ols_all[iter] @ x_test_all[iter].T).T + w0_ols_all[iter]
    cost_OLS_all[iter] = perfs.compute_Oracel_Cost(arcs, grid,cost_dem,c_test_all[iter])
    # cost_OLS_all[iter] = perfs.compute_Cost_with_Prediction(arcs,w0_ols_all[iter],W_ols_all[iter], grid,c_test_all[iter],x_test_all[iter])

In [13]:
# from Shortest_Path_Model import My_ShortestPathModel
# full_shortest_model = My_ShortestPathModel()
# cost_OLS_avg_log = np.zeros(len(iteration_all))
# for iter in iteration_all:
#     # compute OLS performance
#     W_ols, w0_ols, t_ols, obj_ols = ols_method_obj.ols_solver("",x_train_all[iter], np.log(c_train_all[iter]))
#     dem_coef = np.exp((W_ols @ x_test_all[iter].T).T + w0_ols)
#     cost_OLS_log = perfs.compute_Oracel_Cost(arcs, grid,dem_coef,c_test_all[iter])
#     cost_OLS_avg_log[iter] = np.nanmean(cost_OLS_log)
# print("OLS cost = ",np.round(cost_OLS_avg_log,4))

### DDR performance

In [14]:
def obtain_DDR_out_of_sample_performance(mu_all,lamb_all,num_nodes,x_train,c_train,x_test,c_test,perfs,grid,is_log):
    lamb_index = 0
    cost_DDR = {}; w0_ddr_dict = {}; W_ddr_dict = {}
    cost_DDR_avg = np.zeros((len(mu_all),len(lamb_all)))
    mu_index = 0
    for mu in mu_all:
        lamb_index = 0
        for lamb in lamb_all:
            if is_log:
                w0_ddr_dict[mu,lamb],W_ddr_dict[mu,lamb],alpha_rst,obj_ddr = solve_DDR(arcs,lamb,mu,num_nodes,x_train,np.log(c_train))
                cost_dem = np.exp((W_ddr_dict[mu,lamb] @ x_test.T).T + w0_ddr_dict[mu,lamb])
            else:
                w0_ddr_dict[mu,lamb],W_ddr_dict[mu,lamb],alpha_rst,obj_ddr = solve_DDR(arcs,lamb,mu,num_nodes,x_train,c_train)
                cost_dem = (W_ddr_dict[mu,lamb] @ x_test.T).T + w0_ddr_dict[mu,lamb]
            cost_DDR[mu,lamb] = perfs.compute_Oracel_Cost(arcs, grid,cost_dem,c_test)
            # cost_DDR[mu,lamb] = perfs.compute_Cost_with_Prediction(arcs,w0_ddr_dict[mu,lamb],W_ddr_dict[mu,lamb], grid,c_test,x_test)
            cost_DDR_avg[mu_index,lamb_index] = np.nanmean(cost_DDR[mu,lamb])
            lamb_index = lamb_index + 1
        # print("cost_DDR_avg=",np.round(cost_DDR_avg[0,:],4))
        mu_index = mu_index + 1
    return cost_DDR,w0_ddr_dict,W_ddr_dict,cost_DDR_avg

In [15]:
def obtain_ddr_rst(iteration_all,mu_all,lamb_all,num_nodes,x_train_all,c_train_all,x_test_all,c_test_all,perfs,grid,is_log):
    cost_DDR_all = {}; w0_ddr_all = {}; W_ddr_all = {}
    cost_DDR_avg_all = {}
    for iter in iteration_all:
        cost_DDR_all[iter],w0_ddr_all[iter],W_ddr_all[iter],cost_DDR_avg_all[iter] = obtain_DDR_out_of_sample_performance(mu_all,lamb_all,num_nodes,x_train_all[iter],c_train_all[iter],x_test_all[iter],c_test_all[iter],perfs,grid,is_log)
        print("iter = ",iter,",cost_DDR_avg_all = ")
        print(np.round(cost_DDR_avg_all[iter],4))
    # print("regred_DDR_vs_OLS_all=",regred_DDR_vs_OLS_all)
    return cost_DDR_all,w0_ddr_all,W_ddr_all,cost_DDR_avg_all

In [16]:
is_log = False
mu_all = np.arange(0.1,1.0,0.1)
lamb_all = np.round(np.arange(0.0,1.0,0.1),4)
cost_DDR_all,w0_ddr_all,W_ddr_all,cost_DDR_avg_all = obtain_ddr_rst(iteration_all,mu_all,lamb_all,num_nodes,x_train_all,c_train_all,x_test_all,c_test_all,perfs,grid,is_log)

iter =  0 ,cost_DDR_avg_all = 
[[26.8425 26.8389 26.8454 26.8513 26.8539 26.8478 26.8435 26.8454 26.8417
  26.8334]
 [26.8425 26.8373 26.8383 26.8348 26.8344 26.8305 26.837  26.8425 26.8366
  26.8318]
 [26.8425 26.8359 26.835  26.8349 26.8335 26.836  26.836  26.8352 26.8334
  26.8282]
 [26.8425 26.8398 26.8358 26.8354 26.8362 26.8355 26.8385 26.8381 26.8335
  26.83  ]
 [26.8425 26.8425 26.8361 26.8325 26.8344 26.8378 26.8372 26.8401 26.8406
  26.8418]
 [26.8425 26.8412 26.8387 26.8309 26.8298 26.8303 26.8309 26.8352 26.8355
  26.8339]
 [26.8425 26.8403 26.8373 26.8336 26.8322 26.8281 26.8271 26.8252 26.8234
  26.8234]
 [26.8425 26.8418 26.8394 26.8399 26.8392 26.8365 26.8357 26.8339 26.8319
  26.8305]
 [26.8425 26.8435 26.8422 26.8398 26.8402 26.8396 26.8402 26.8392 26.8397
  26.8394]]
iter =  1 ,cost_DDR_avg_all = 
[[26.6909 26.6898 26.6939 26.6964 26.7053 26.7109 26.7115 26.7217 26.7431
  26.7569]
 [26.6909 26.6936 26.6951 26.6932 26.6941 26.6934 26.6984 26.7052 26.7128
  26.721 ]
 [

In [17]:
regret_ddr_vs_ols_all = {}; regret_ddr_vs_ols_avg = np.zeros((len(mu_all),len(lamb_all)))

cost_OLS_avg_all = np.zeros(len(iteration_all))
cost_Oracle_avg_all = np.zeros(len(iteration_all))

for iter_index in range(len(iteration_all)):
    iter = iteration_all[iter_index]
    cost_OLS_avg_all[iter_index] = np.nanmean(cost_OLS_all[iter])
    cost_Oracle_avg_all[iter_index] = np.nanmean(cost_Oracle_all[iter])

    regret_ddr_vs_ols_all[iter_index] = (cost_OLS_avg_all[iter_index] - cost_DDR_avg_all[iter])/np.abs(cost_OLS_avg_all[iter_index] - cost_Oracle_avg_all[iter_index])
    regret_ddr_vs_ols_avg = regret_ddr_vs_ols_avg + regret_ddr_vs_ols_all[iter_index]
regret_ddr_vs_ols_avg = regret_ddr_vs_ols_avg/len(iteration_all)
print("regret_ddr_vs_ols_avg")
print(np.round(regret_ddr_vs_ols_avg*100,2))

regret_ddr_vs_ols_avg
[[ 0.000e+00  4.000e-02 -2.910e+00 -6.680e+00 -1.168e+01 -1.534e+01
  -1.783e+01 -2.568e+01 -3.352e+01 -4.260e+01]
 [ 0.000e+00 -5.200e-01 -2.840e+00 -4.630e+00 -8.130e+00 -1.000e+01
  -1.195e+01 -1.747e+01 -2.339e+01 -3.273e+01]
 [ 0.000e+00 -1.610e+00 -2.330e+00 -4.760e+00 -4.630e+00 -5.500e+00
  -8.110e+00 -9.280e+00 -1.314e+01 -1.407e+01]
 [ 0.000e+00 -1.500e-01 -3.000e-01 -4.080e+00 -5.040e+00 -7.250e+00
  -9.710e+00 -1.056e+01 -1.350e+01 -1.686e+01]
 [ 0.000e+00 -5.000e-01 -1.030e+00 -7.400e-01 -2.070e+00 -3.220e+00
  -4.150e+00 -5.730e+00 -8.270e+00 -1.002e+01]
 [ 0.000e+00 -2.030e+00 -1.990e+00 -8.800e-01 -8.700e-01 -3.770e+00
  -4.480e+00 -6.480e+00 -6.800e+00 -6.670e+00]
 [ 0.000e+00 -9.000e-01 -3.700e-01 -8.000e-01  7.200e-01  9.500e-01
   2.400e-01 -1.320e+00 -2.550e+00 -2.880e+00]
 [ 0.000e+00 -2.500e-01 -1.300e-01  4.000e-02  3.900e-01  4.500e-01
   3.000e-02 -1.300e+00 -7.100e-01 -8.700e-01]
 [ 0.000e+00 -2.600e-01  2.400e-01  8.100e-01  7.100e-01  

In [37]:
print(np.round(regret_ddr_vs_ols_avg[8,:]*100,2))

[ 0.   -0.26  0.24  0.81  0.71  0.79  0.77  0.86  0.98  0.72]


In [18]:
# with open(DataPath+'cost_OLS_all.pkl', "wb") as tf:
#     pickle.dump(cost_OLS_all,tf)
# with open(DataPath+'cost_Oracle_all.pkl', "wb") as tf:
#     pickle.dump(cost_Oracle_all,tf)
# with open(DataPath+'cost_DDR_all.pkl', "wb") as tf:
#     pickle.dump(cost_DDR_all,tf)

In [19]:
# seed = 1
# lamb = 1
# cost_OLS_esti = x_test_all[seed] @ np.asarray(W_ols_all[seed]).T + w0_ols_all[seed]
# cost_DDR_esti = x_test_all[seed] @ np.asarray(W_ddr_all[seed,mu][lamb]).T + w0_ddr_all[seed,mu][lamb]

# S_test_index = 3
# print("cost_DDR_esti=",np.round(cost_DDR_esti[S_test_index,],2))
# # print("cost:(0,1)-->(1,3) = ",cost_DDR_esti[S_test_index,0] + cost_DDR_esti[S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",cost_DDR_esti[S_test_index,1] + cost_DDR_esti[S_test_index,3])

# print("cost_OLS_esti=",np.round(cost_OLS_esti[S_test_index,],2))
# # print("cost:(0,1)-->(1,3) = ",cost_OLS_esti[S_test_index,0] + cost_OLS_esti[S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",cost_OLS_esti[S_test_index,1] + cost_OLS_esti[S_test_index,3])


# print("cost_Oracle=",np.round(c_test_all[seed][S_test_index],2))
# # print("cost:(0,1)-->(1,3) = ",c_test_all[seed][S_test_index,0] + c_test_all[seed][S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",c_test_all[seed][S_test_index,1] + c_test_all[seed][S_test_index,3])

# Plot figures

In [20]:
def cross_compare2plus(c_item, c_base, c_oracle):
    c_item = np.asarray(c_item)
    c_base = np.asarray(c_base)
    c_oracle = np.asarray(c_oracle)

    N = len(c_item)
    c_diff = c_item - c_base
    lbel = np.zeros((N,1))
    
    equals = np.sum(c_diff == 0)
    wins = np.sum(c_diff < 0)
    lose = np.sum(c_diff > 0)
    
    lbel[c_diff < 0] = 1
    lbel[c_diff > 0] = -1
    
    # print("Num_train =",N,",Num_equals =",equals,",Num_wins =",wins,",Num_lose =",lose)
    # print("base cost = ", np.mean(c_base),",item cost = ",np.mean(c_item))
    if N == equals:
        win_ratio = 0.5
    else:
        win_ratio = wins/(N - equals)
    # cost_reduction = (np.mean(c_base) - np.mean(c_item) )/np.abs(np.mean(c_oracle))
    regret_reduction = (np.nanmean(c_base) - np.nanmean(c_item))/np.abs(np.nanmean(c_base) - np.nanmean(c_oracle))
    return lbel, win_ratio, regret_reduction

### DDR figure

In [21]:
# H2H_DDR_vs_OLS_all = {}; regret_reduction_DDR_vs_OLS_all = {}
# for mu in mu_all:
#     for lamb in lamb_all:
#         H2H_DDR_vs_OLS_arr = np.zeros(len(iteration_all)); regret_reduction_DDR_vs_OLS_arr = np.zeros(len(iteration_all))
#         # print("lamb = ",lamb)
#         for iter in iteration_all:
#             lbel, H2H_DDR_vs_OLS_arr[iter-1], regret_reduction_DDR_vs_OLS_arr[iter-1] = cross_compare2plus(cost_DDR_all[iter][mu,lamb],cost_OLS_all[iter], cost_Oracle_all[iter])
#         H2H_DDR_vs_OLS_all[mu,lamb] = H2H_DDR_vs_OLS_arr; regret_reduction_DDR_vs_OLS_all[mu,lamb] = regret_reduction_DDR_vs_OLS_arr

In [22]:
# mu = mu_all[0]
# lamb = lamb_all[7]
# import Figures
# file_name = DataPath + "figure_mu="+str(mu)+"_lamb="+str(lamb)
# Figures.figure_plot_upleft(H2H_DDR_vs_OLS_all[mu,lamb]*100, regret_reduction_DDR_vs_OLS_all[mu,lamb], file_name, size = (5, 5), move = [-0.12, 0.04, 0.35, 0.55], 
#                     ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

In [23]:
# import matplotlib.pyplot as plt
# fig = plt.figure()
# ax = plt.subplot()
# ax.plot(lamb_all,regret_all[seed,mu])

### SPO figure

In [24]:
# H2H_SPO_vs_OLS_arr = np.zeros(len(seed_all)); regret_reduction_SPO_vs_OLS_arr = np.zeros(len(seed_all))
# seed_index = 0
# for seed in seed_all:
#     lbel, H2H_SPO_vs_OLS_arr[seed_index], regret_reduction_SPO_vs_OLS_arr[seed_index] = cross_compare2plus(cost_EPO_all[seed]["SPO"],cost_OLS_all[seed], cost_Oracle_all[seed])
#     seed_index = seed_index + 1
# H2H_SPO_vs_OLS_all = H2H_SPO_vs_OLS_arr; regret_reduction_SPO_vs_OLS_all = regret_reduction_SPO_vs_OLS_arr

In [25]:
# Figures.figure_plot_upleft(H2H_SPO_vs_OLS_all*100, regret_reduction_SPO_vs_OLS_all, "", size = (5, 5), move = [-0.12, 0.04, 0.35, 0.55], 
#                     ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

In [26]:
# regret_reduction_SPO_vs_OLS_arr